In [1]:
!nvidia-smi

Fri Nov 14 15:41:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# !unzip -q /content/labeled_data.zip -d /content/custom_data

In [3]:
# !wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# # TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
# !python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00


In [5]:
with open('/content/drive/MyDrive/DocLayNet_filtered/classes.txt', 'r') as f:
  label = f.readlines()
  print(label)

['Caption\n', 'PageNumber\n', 'Picture']


In [6]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/drive/MyDrive/DocLayNet_filtered',
      'train': 'train/images',
        'val': 'test/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/drive/MyDrive/DocLayNet_filtered/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

Created config file at /content/data.yaml

File contents:

path: /content/drive/MyDrive/DocLayNet_filtered
train: train/images
val: test/images
nc: 3
names:
- Caption
- PageNumber
- Picture


In [7]:
!yolo detect train data=/content/data.yaml model=yolov8s.pt epochs=40 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj

In [8]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=drive/MyDrive/DocLayNet_filtered/test/images save=True save_txt=True save_conf=True

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs

image 1/1200 /content/drive/MyDrive/DocLayNet_filtered/test/images/04eef95362daf7932b8b3946e3585f3a0cdf18379bd19754051ecc4ac01723fa.png: 640x640 2 Captions, 2 PageNumbers, 2 Pictures, 16.3ms
image 2/1200 /content/drive/MyDrive/DocLayNet_filtered/test/images/07008bc020f888555801b65767472d16d89df20d8d2b58d3c3dc427ba4847663.png: 640x640 1 Caption, 1 Picture, 16.3ms
image 3/1200 /content/drive/MyDrive/DocLayNet_filtered/test/images/0a543e68dbb5dd68ea965012786b3d07a1d32018f4dfe9f9016c62428ca38d0d.png: 640x640 2 PageNumbers, 16.3ms
image 4/1200 /content/drive/MyDrive/DocLayNet_filtered/test/images/0ece30e5130c582770d840243c11dfcde0e34d5eb2b6ce0f4209e5a0e55a790c.png: 640x640 1 PageNumber, 16.3ms
image 5/1200 /content/drive/MyDrive/DocLayNet_filtered/test/images/1235bb65aef4212fd02a4e433eb6617c319c8bef2bad99a41e7e0712d70fa9c1.png:

In [9]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

Output hidden; open in https://colab.research.google.com to view.

In [10]:
import os
import glob
from PIL import Image

# Paths
label_dir = '/content/runs/detect/predict/labels'
image_dir = '/content/runs/detect/predict'

for label_path in glob.glob(os.path.join(label_dir, '*.txt')):
    image_name = os.path.splitext(os.path.basename(label_path))[0] + '.jpg'
    image_path = os.path.join(image_dir, image_name)

    img = Image.open(image_path)
    img_width, img_height = img.size

    with open(label_path, 'r') as f:
        lines = f.readlines()

    print(f"Image: {image_name}")
    for line in lines:
        print(line, end='')
        cls, x_center, y_center, w, h, confidence = map(float, line.strip().split())

        # Convert normalized → pixel coordinates
        x_min = int((x_center - w / 2) * img_width)
        y_min = int((y_center - h / 2) * img_height)
        x_max = int((x_center + w / 2) * img_width)
        y_max = int((y_center + h / 2) * img_height)

        print(f"Class {int(cls)}: (x_min={x_min}, y_min={y_min}), (x_max={x_max}, y_max={y_max})")
        print(f"Image width: {img_width}, image height: {img_height}")
        print()
    print("=====================================================================")


Image: d043d8dcc8bed3d1512bef6294d92c3c724523e996656455b2c2eb07c4787d7d.jpg
1 0.499735 0.90562 0.0190141 0.0146952 0.865102
Class 1: (x_min=502, y_min=920), (x_max=521, y_max=935)
Image width: 1025, image height: 1025

Image: 1bbda3f6ad2fe5d3b0b64bd8450e5cd41644f416f71eb2d15f1b928893fd37b9.jpg
1 0.489085 0.962295 0.0101822 0.0151512 0.820158
Class 1: (x_min=496, y_min=978), (x_max=506, y_max=994)
Image width: 1025, image height: 1025

Image: c77d6efe7fc4723076f84558a0604e1d7e60051f023d531984048c15770d78e4.jpg
1 0.500004 0.905855 0.0286493 0.0151472 0.88476
Class 1: (x_min=497, y_min=920), (x_max=527, y_max=936)
Image width: 1025, image height: 1025

Image: f6d75798062b25245b314a881078ddb355821b7397f9abc57f1a05eec14c6ad4.jpg
0 0.289807 0.792506 0.403877 0.182342 0.931166
Class 0: (x_min=90, y_min=718), (x_max=504, y_max=905)
Image width: 1025, image height: 1025

2 0.505082 0.203471 0.804223 0.247956 0.909011
Class 2: (x_min=105, y_min=81), (x_max=929, y_max=335)
Image width: 1025, imag

In [11]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/finetuned_yolo_doclaynet
!cp /content/runs/detect/train/weights/best.pt /content/finetuned_yolo_doclaynet/finetuned_yolo_doclaynet.pt
!cp -r /content/runs/detect/train /content/finetuned_yolo_doclaynet
!cp -r /content/runs/detect/predict /content/finetuned_yolo_doclaynet

# Zip into "finetuned_yolo_doclaynet.zip"
%cd finetuned_yolo_doclaynet
!zip /content/finetuned_yolo_doclaynet.zip finetuned_yolo_doclaynet.pt
!zip -r /content/finetuned_yolo_doclaynet.zip train
!zip -r /content/finetuned_yolo_doclaynet.zip predict
%cd /content

/content/finetuned_yolo_doclaynet
  adding: finetuned_yolo_doclaynet.pt (deflated 8%)
  adding: train/ (stored 0%)
  adding: train/BoxR_curve.png (deflated 11%)
  adding: train/val_batch0_pred.jpg (deflated 16%)
  adding: train/val_batch0_labels.jpg (deflated 16%)
  adding: train/results.csv (deflated 61%)
  adding: train/BoxF1_curve.png (deflated 10%)
  adding: train/args.yaml (deflated 53%)
  adding: train/train_batch0.jpg (deflated 7%)
  adding: train/train_batch11252.jpg (deflated 16%)
  adding: train/train_batch11251.jpg (deflated 12%)
  adding: train/train_batch11250.jpg (deflated 15%)
  adding: train/val_batch1_pred.jpg (deflated 14%)
  adding: train/val_batch2_pred.jpg (deflated 15%)
  adding: train/val_batch2_labels.jpg (deflated 16%)
  adding: train/results.png (deflated 6%)
  adding: train/confusion_matrix.png (deflated 28%)
  adding: train/weights/ (stored 0%)
  adding: train/weights/best.pt (deflated 8%)
  adding: train/weights/last.pt (deflated 8%)
  adding: train/train_b